In [19]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

costsheet = pd.read_csv('costs.csv') 
costsheet['Mendenhall'] = pd.to_numeric(costsheet['Mendenhall'], errors='coerce') #csv having some entries in these columns as strings, fix before submission
costsheet['Caldwell'] = pd.to_numeric(costsheet['Caldwell'], errors='coerce')
costsheet['Pomerene'] = pd.to_numeric(costsheet['Pomerene'], errors='coerce')
costsheet['friend1'] = pd.to_numeric(costsheet['friend1'], errors='coerce')
costsheet['friend2'] = pd.to_numeric(costsheet['friend2'], errors='coerce')
costsheet['friend3'] = pd.to_numeric(costsheet['friend3'], errors='coerce')

In [23]:
## HANGOUTS PER MONTH * OPPORTUNITY COST * PROBABILITY ; FOR EACH FRIEND
def friendcost(id):
    return 6*11*(0.025*(costsheet['friend1'][id]/.1)) + 6*18*(0.025*(costsheet['friend2'][id]/.1)) + 6*48*(0.025*(costsheet['friend3'][id]/.1))

## NUMBER OF CLASSES * TUITION PER CLASS * PROBABILITY OF MISSING CLASS, DEPENDENT ON BUYING A LIMEPRIME SUBSCRIPTION
def class_cost(id):
    return 12*51.4*.05*(closest_class(id)/.25) * (costsheet['limeprime'][id] / 3)

## which first class option is closest; for lime scooter purposes 
def closest_class(id):
    return costsheet['mindist'][id]
    
## BASE COST OF EACH RENTAL
def basecost(id):
    cost = costsheet['Rent'][id] #rent
    cost = cost + 25 * costsheet['Water'][id] + 50 * costsheet['Electric'][id] + 30 * costsheet['Gas'][id]
    cost = cost + 20 * costsheet['Internet'][id] + 24 * costsheet['Laundry'][id] # utilities
    cost = cost + 13.68 * costsheet['Emergencies'][id] + 6.84 * costsheet['Generals'][id] #landlords
    cost = cost + 13.68 * costsheet['Safety'][id] + 4.16 * costsheet['Sidewalks'][id] + 3.9 * costsheet['Clean'][id]
    cost = cost + costsheet['Off-Street'][id] * costsheet['Fee'][id] #parking

    return float(cost)

def limecost(id):
    return costsheet['limeprime'][id] * (6 + 24 * 0.3 * closest_class(id))

m = gp.Model("grpprj")
m.Params.LogToConsole = 0

## Initialize 12 house choices
x = m.addVars(12, vtype=GRB.BINARY, name= "x")

#cost of rent, utilities, landlord costs, + friend opportunity costs, + class missing opportunity costs, + limeprime costs
m.setObjective(
    x[0] * (basecost(1) + friendcost(1) + class_cost(1) + limecost(1)) + 
    x[1] * (basecost(2) + friendcost(2) + class_cost(2) + limecost(2)) + 
    x[2] * (basecost(3) + friendcost(3) + class_cost(3) + limecost(3)) + 
    x[3] * (basecost(4) + friendcost(4) + class_cost(4) + limecost(4)) + 
    x[4] * (basecost(5) + friendcost(5) + class_cost(5) + limecost(5)) + 
    x[5] * (basecost(6) + friendcost(6) + class_cost(6) + limecost(6)) + 
    x[6] * (basecost(7) + friendcost(7) + class_cost(7) + limecost(7)) + 
    x[7] * (basecost(8) + friendcost(8) + class_cost(8) + limecost(8)) + 
    x[8] * (basecost(9) + friendcost(9) + class_cost(9) + limecost(9)) + 
    x[9] * (basecost(10) + friendcost(10) + class_cost(10) + limecost(10)) + 
    x[10] * (basecost(11) + friendcost(11) + class_cost(11) + limecost(11)) + 
    x[11] * (basecost(12) + friendcost(12) + class_cost(12) + limecost(12)),
GRB.MINIMIZE)

## Ensure sum of apartment decisions is equal to one
m.addConstr(x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] == 1)

m.optimize()

num = 6
for i, var in enumerate(m.getVars()):
    if var.x == 1:
        num = 6

print('The recommended choice is', costsheet['address'][num], 'with a total sum of monthly costs and opportunity costs of', round(m.objVal,2))
print('With this property, Jake will schedule his class at', costsheet.columns[costsheet.iloc[num] == closest_class(num)].tolist()[0], 'first.' )
if costsheet['limeprime'][num] == 1:
    print('Jake will purchase a LimePrime subscription.')
else:
    print('Jake will not purchase a LimePrime subscription.')


The recommended choice is 113 E Frambes Ave with a total sum of monthly costs and opportunity costs of 733.32
With this property, Jake will schedule his class at Mendenhall first.
Jake will not purchase a LimePrime subscription.


In [24]:
#costs for each property
for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
    print(basecost(i) + friendcost(i) + class_cost(i) + limecost(i))

#cost for optimal property
print("obj_func = ", m.objVal)
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

1040.5695294226
894.0559051708
1194.8135294226001
1058.4495294225999
923.4255555513798
1280.2559051708
733.3159999908798
926.0695294225999
1073.50422895052
1257.14822895052
989.0362289505201
876.7313507307399
obj_func =  733.3159999908798
x[0] = 0
x[1] = 0
x[2] = 0
x[3] = 0
x[4] = 0
x[5] = 0
x[6] = 1
x[7] = 0
x[8] = 0
x[9] = 0
x[10] = 0
x[11] = 0
